# Project Background

### The business objective of this project is to make three recommendations to our company who wants to create a new movie studio to compete with competitors. We are charged with finding out what movies are doing the best at the box office and to help our company decide what kind of films to create.

### To help assist us, we used data from (INSERT DATA HERE). (EXPLAIN DATA, POSITIVES AND NEGATIVES)

# Importing Libraries And Reading in Data

In [186]:
import pandas as pd
import numpy as np
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [187]:
#read in csv file, TheNumbers
df_movie_budgets = pd.read_csv('data/tn.movie_budgets.csv.gz') #5782 x 6, budget, domestic_gross, worldwide_gross from TheNumbers

#read in sql file, IMDB
conn = sqlite3.connect("data/im.db/im.db")
df_imdb = pd.read_sql( #682303 × 6 runtime, genre, actors/directors from IMDB
    """
    SELECT basic.primary_title as movie_title, basic.start_year as year, basic.runtime_minutes, basic.genres, 
    persons.primary_name as person_name, persons.primary_profession
    FROM movie_basics as basic
    JOIN principals
    ON basic.movie_id = principals.movie_id
    JOIN persons
    ON principals.person_id = persons.person_id
    WHERE persons.primary_profession LIKE '%actor%' OR persons.primary_profession LIKE '%actress%' OR persons.primary_profession LIKE '%director%'
    """
, conn
).drop_duplicates()

In [188]:
#new column to track release year to help with the merge
df_movie_budgets["release_year"] = [int(data[-4:]) for data in df_movie_budgets["release_date"]]
#merging TheNumbers and IMDB on movie and year to be sure that different movies with the same name dont get merged together
df_final = pd.merge(df_movie_budgets, df_imdb, left_on=['movie', 'release_year'], right_on=['movie_title', 'year'])

In [189]:
df_final = df_final.drop(columns=['year', 'movie_title', 'id'])
df_final

,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year,runtime_minutes,genres,person_name,primary_profession
0,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy",Geoffrey Rush,"actor,soundtrack,producer"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy",Terry Rossio,"writer,director,producer"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy",Ian McShane,"actor,director,producer"
3,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy",Stuart Beattie,"writer,director,producer"
4,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011,136.0,"Action,Adventure,Fantasy",Johnny Depp,"actor,producer,soundtrack"
...,...,...,...,...,...,...,...,...,...,...
10743,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0,2014,60.0,Drama,Mugambi Nthiga,"actor,writer,casting_department"
10744,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0,2014,60.0,Drama,Paul Ogola,actor
10745,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0,2014,60.0,Drama,Tim Mutungi,actor
10746,"Dec 31, 2014",Stories of Our Lives,"$15,000",$0,$0,2014,60.0,Drama,Kelly Gichohi,actress


# Formatting Change and Filtering

In [196]:
#removing NA values from genres column
df_final = df_final.loc[df_final["genres"].isna() == False]
df_final["production_budget_int"] = [int(budget.strip('$').replace(",", "")) for budget in df_final["production_budget"]]
#df_final["domestic_gross_int"] = [int(budget.strip('$').replace(",", "")) for budget in df_final["domestic_gross"]]
df_final["worldwide_gross_int"] = [int(budget.strip('$').replace(",", "")) for budget in df_final["worldwide_gross"]]
df_final["profit"] = df_final["worldwide_gross_int"] - df_final["production_budget_int"]

<ipython-input-196-af94fe305fa9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["production_budget_int"] = [int(budget.strip('$').replace(",", "")) for budget in df_final["production_budget"]]
<ipython-input-196-af94fe305fa9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["worldwide_gross_int"] = [int(budget.strip('$').replace(",", "")) for budget in df_final["worldwide_gross"]]
<ipython-input-196-af94fe305fa9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [197]:
df_final = df_final.loc[df_final["profit"] > 100000000]
df_final = df_final.drop(df_final.loc[df_final["movie"] == "Hercules"].loc[df_final["genres"] == "Comedy"].index, axis='index')
df_final = df_final.drop(df_final.loc[df_final["movie"] == "Cinderella"].loc[df_final["genres"] == "Fantasy"].index, axis='index')
df_final = df_final.drop(df_final.loc[df_final["movie"] == "Coco"].loc[df_final["genres"] == "Horror"].index, axis='index')
df_final = df_final.drop(df_final.loc[df_final["movie"] == "Alice in Wonderland"].loc[df_final["genres"] == "Fantasy,Musical"].index, axis='index')
df_final = df_final.drop(df_final.loc[df_final["movie"] == "The Artist"].loc[df_final["genres"] == "Thriller"].index, axis='index')

In [198]:
print(df_final[["movie", "profit", "genres"]].drop_duplicates().sort_values(["profit"], ascending=False))
def avg_profit_genre(genre_name):
    genre_combined_profit = 0
    genre_count = 0
    for movie in df_final[["genres", "profit"]].drop_duplicates().values:
        if genre_name in movie[0]:
            genre_combined_profit += movie[1]
            genre_count += 1
    return math.ceil(genre_combined_profit/genre_count)
genre_averages = {"Action": avg_profit_genre("Action"), "Adventure": avg_profit_genre("Adventure"), 
              "Animation": avg_profit_genre("Animation"), "Biography": avg_profit_genre("Biography"), 
              "Comedy": avg_profit_genre("Comedy"), "Crime": avg_profit_genre("Crime"), 
              "Documentary": avg_profit_genre("Documentary"), "Drama": avg_profit_genre("Drama"),
              "Fantasy": avg_profit_genre("Fantasy"), "Family": avg_profit_genre("Family"),
              "History": avg_profit_genre("History"), "Horror": avg_profit_genre("Horror"),
              "Music": avg_profit_genre("Music"), "Musical": avg_profit_genre("Musical"),
              "Mystery": avg_profit_genre("Mystery"), "Sci-Fi": avg_profit_genre("Sci-Fi"),
              "Thriller": avg_profit_genre("Thriller"), "War": avg_profit_genre("War"),
              "Western": avg_profit_genre("Western")}
genre_averages

                       movie      profit                   genres
21    Avengers: Infinity War  1748134200  Action,Adventure,Sci-Fi
170           Jurassic World  1433854864  Action,Adventure,Sci-Fi
338                Furious 7  1328722794    Action,Crime,Thriller
131             The Avengers  1292935897  Action,Adventure,Sci-Fi
204            Black Panther  1148258224  Action,Adventure,Sci-Fi
...                      ...         ...                      ...
7851      The Purge: Anarchy   102534881     Action,Horror,Sci-Fi
7057        Think Like a Man   101373764           Comedy,Romance
2675              Goosebumps   100905324  Adventure,Comedy,Family
9941        The Devil Inside   100759490                   Horror
2994          Into the Storm   100429371   Action,Adventure,Drama

[385 rows x 3 columns]


{'Action': 379421200,
 'Adventure': 418484478,
 'Animation': 399765177,
 'Biography': 224641882,
 'Comedy': 301379412,
 'Crime': 233587409,
 'Documentary': 183994263,
 'Drama': 226631231,
 'Fantasy': 393035119,
 'Family': 367543354,
 'History': 173171337,
 'Horror': 203598497,
 'Music': 380573961,
 'Musical': 700932628,
 'Mystery': 217980477,
 'Sci-Fi': 461207145,
 'Thriller': 283956942,
 'War': 156043081,
 'Western': 283612626}

# Exploratory Data Analysis

In [13]:
#plt.scatter(df_final["id"], df_final["worldwide_gross"])
#plt.show()

# Write to File

# Appendix

In [40]:
df_movie_gross = pd.read_csv('data/bom.movie_gross.csv.gz') #3387 x 5 domestic_gross, foreign_gross, year from
df_movie_info = pd.read_csv('data/rt.movie_info.tsv.gz', sep='\t') #1560 x 12 rating, genre, director, writer, box office, runtime from Rotten Tomatoes
df_reviews = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='unicode_escape') #54432 x 8 rating, fresh, top critic from Rotten tomatoes
df_movies = pd.read_csv('data/tmdb.movies.csv.gz') #26517 x 10 popularity, vote_average, vote_count #from TheMoviesTB
df_directors = pd.read_sql(
    """
    Select persons.primary_name, persons.primary_profession, movie_basics.primary_title
    FROM directors
    JOIN persons
    ON directors.person_id = persons.person_id
    JOIN movie_basics
    ON movie_basics.movie_id = directors.movie_id
    """
, conn
)
df_persons = pd.read_sql(
    """
    SELECT *
    FROM known_for
    JOIN persons
    ON known_for.person_id = persons.person_id
    """
, conn
)
df_imdb[df_imdb["person_name"] == "James Cameron"] #Something we noticed is that Avatar is not a part of the IMDB data
df_imdb["year"].value_counts() #The data seems to start in 2010 and starts dwindling in 2020

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
